### import tensorflow and packages, then import keras from tensorflow

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.python import keras
from PIL import Image


from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from tensorflow.python.keras.preprocessing import image

import os
import shutil

#check all versions
print('tensorflow version is', tf.__version__, ) #should be >= 1.11.0 
print('keras version is', keras.__version__) #shoudl be >= 2.1.6-tf
print('numpy version is', np.__version__) #should be >= 1.15.3

ModuleNotFoundError: No module named 'tensorflow.python.keras.applications.vgg16'

###### since we have a small dataset, we can use a pre-trained network that has already been trained on a large-scale image classification task. 

In [26]:
#let's use a convnet trained on the ImageNet dataset (1.4e6 labeled images, and 1000 clases); we will be particularly basing out model from a binary classification between dogs and cats
conv_base = VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(150,150,3))
#weights - to specify which weight checkpoint to initialize the model from
#include_top - we set to False because this is a DNN top layer default for 1000 classes, but we only need it for our own DNN with only 2 classes
#input_shape - shape of image tensors that we will feed into the network. Argument is not important.

58892288/58889256 [==============================] - 9s 0us/step


#### didactic aside: this is the convolutional base of the standard VGG16

In [29]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

#### we can see that  this base has ~15e6 parameters aka yuge. 
not only that but it's an immense amount of layers
I can't get pictures to work on here atm, but imagine all these layers being a long vertical column (trained convolutional base) which feeds into the trained classifier to produce predictions. 
convnets generally work in two parts: input layer which feeds into a conv layer which is pooled until the next conv layer which is pooled. 
As you can see the dimensions ever decrease with each maxpool, which would make more sense in image form (I guess I'll add that to the read me)

#### either way, we exploit this by running new data (our data) through this conv base, and simply replacing the trained classifier
basically, the idea is to get as far away from this DNN as possible. 
it's reusable and generalizable because of the process of taking in data in the earlier layers which extract local and highly generic feature maps (visual edges, colors, textures) while layers higher up extract more abstract concepts (such as cat ear or dog eye, or in our case colormaps). 

#### let's do two things: 1. running the convnet as is over our dataset, recording its output to a numpy array (on disk), use this data as input to a standalone densely-connected classifier. there's no leverage here though, so I may skip this for now. ; 2. extending the model we have (conv base) by adding a dense layer to the top.

## Build Keras Model
### let's just leverage this 
you will also see a model summary, which you can compare to the earlier one.

In [51]:
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
conv_base.trainable = False

In [52]:
#let's check it out
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_5 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)               2097408   
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 257       
Total params: 16,812,353
Trainable params: 2,097,665
Non-trainable params: 14,714,688
_________________________________________________________________


### check it out we have 2mil trainable params - great!
### let's now apply this Keras model to our TF estimator

In [53]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

In [54]:
#model_dir will be our locationt o store trained tensorflow models. We will view this progress using tensorboard (hopefully). 
model_dir = os.path.join(os.getcwd(), "models//catvsdog").replace("//", "\\")
os.makedirs(model_dir, exist_ok=True)
print("model_dir: ",model_dir)
est_catvsdog = tf.keras.estimator.model_to_estimator(keras_model=model,
                                                    model_dir=model_dir)

model_dir:  C:\Users\Scott\tensorflowstuffadnan\Nuke the rest - new beginning\models\catvsdog
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Scott\\tensorflowstuffadnan\\Nuke the rest - new beginning\\models\\catvsdog', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002A897EC2630>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master

In [55]:
# input layer name
input_name = model.input_names[0]
input_name
#issues here, msot likely because using same function names for both examples

'vgg16_input'

In [56]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

# Download the Dataset; ##let's just do a trial with cats and dogs
this will consist of 3 subsets: 1. training set of 1.6x samples of each class (dog and cat aka Task1 and Task2); 2. test set of x samples of each class. 

In [61]:
#this is where I ran into problems before because this jupyter notebook has
#become awfully cluttered and this particular cell creates directories

#original directory of dataset
original_dataset_dir = 'C:\\Users\\Scott\\AppData\\Local\\Temp\\HamsterArc4\\train\\train'

# The directory where we will
# store our smaller dataset
base_dir = './data/dog_vs_cat_small'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
test_cats_dir = os.path.join(test_dir, 'cats')
test_dogs_dir = os.path.join(test_dir, 'dogs')

In [62]:
os.makedirs(base_dir, exist_ok=True)
# Directories for our training,
# validation and test splits
os.mkdir(train_dir)
os.mkdir(test_dir)
# Directory with our training cat pictures
os.mkdir(train_cats_dir)
# Directory with our training dog pictures
os.mkdir(train_dogs_dir)
# Directory with our validation cat pictures
os.mkdir(test_cats_dir)
# Directory with our validation dog pictures
os.mkdir(test_dogs_dir)
# Copy first 1000 cat images to train_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)
# Copy first 1000 dog images to train_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)

# Copy next 500 cat images to test_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)    
    
# Copy next 500 dog images to test_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: './data/dog_vs_cat_small\\train'